<a href="https://colab.research.google.com/github/WaffleCollege/ori/blob/main/04_Flask_3_with_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #1.　ngrokのauth_tokeを設定して前回の内容を実行[事前準備、設定]
# 1,2 Flask, pyngrokのインストール
!pip install flask pyngrok
#  必要なモジュールのimport
import os
# flaskからFlaskクラスをインポート
from flask import Flask
# pyngrokからngrokモジュールとconfモジュールをインポート
from pyngrok import ngrok, conf
# 設定
# ngrokトークンを設定
conf.get_default().auth_token = "2lkOXuufs2PtxUjWJ01xOwRV2rw_5jQeF8VkacBWSMcyB1EuB" #ここに自分のNgrokトークンを入れてください！！！！！

#
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
conf.get_default().auth_token = "2lkOXuufs2PtxUjWJ01xOwRV2rw_5jQeF8VkacBWSMcyB1EuB"

In [2]:
#@title #2.データベースとデータの中身を用意する
import sqlite3

# データベースのテーブル（データを入れるための形のある箱）を用意する
conn = sqlite3.connect('sample_db.sqlite')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER
)
''')

conn.commit()
conn.close()

# 3. HTMLを更新する(drive/My Drive/flask_blog/templates/home.htmlを更新）

```
<!DOCTYPE html>
<html>
  <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css')}}">
<head>
</head>
<body>
    <header>
      <h1>ミニブログをつくってみよう !</h1>
    </header>
    <ul>
       {% for book in books %} <!-- ループさせてそれぞれ要素を取り出す -->
      <li>{{ book }}</li>
       {% endfor %}
</ul>
</body>
</html>
```


In [3]:
#@title　#4.アプリケーションを初期化する

from flask import g, render_template


# テンプレートと静的ファイルの置き場所を定数に代入
TEMPLATE_PATH = 'drive/My Drive/flask_blog/templates'
STATIC_PATH = 'drive/My Drive/flask_blog/static'
# Flaskのインスタンスを作るときにそれを利用してファイルパスを設定する
app = Flask(__name__, template_folder=TEMPLATE_PATH, static_folder=STATIC_PATH)





In [4]:
#@title　#5.アプリケーションとデータベースを繋ぐための関数を用意する

DATABASE = 'sample_db.sqlite'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def query_db(query, args=(), one=False):
    cur = get_db().execute(query, args)
    rv = cur.fetchall()
    cur.close()
    return (rv[0] if rv else None) if one else rv

def insert_or_delete_db(query, args=()):
    get_db().execute(query, args)
    get_db().commit()



In [5]:
#@title #6.データベースにデータを登録する

# アプリケーションが実行されるコンテキスト内で行うという指定のwith
with app.app_context():
  # データが何度も登録されないように一度データベースの中身を削除するSQL文を構成
  delete_sql  ="""
   DELETE FROM books
  """
  # SQL文を実行してデータを削除
  insert_or_delete_db(delete_sql)
  # データをデータベースに登録するSQL文を構成
  insert_sql ="""
    INSERT INTO books (id,title, author, year_published)
    VALUES (1,'わたし×IT＝最強説', 'NPO法人Waffle', 2023),
     (2,'ユウと魔法のプログラミング・ノート', '鳥井雪', 2023),
     (3, 'ハッカーと画家', 'Paul Graham, 川合 史朗', 2005)
    """
  # SQL文を実行してデータを登録
  insert_or_delete_db(insert_sql)

In [6]:
#@title #7.アプリケーションでデータを読み出す+テンプレートにデータを渡すプログラミング

# /（ルート）にアクセスしたら次に書く関数(hello_world)を実行するという宣言
@app.route("/")
def home():
    #データベースからデータを引き出す（配列の配列の形で帰ってくる）
    books = query_db("SELECT * FROM books")

    # テンプレートにbooks変数を読み出す
    return render_template('home.html', books = books)

In [7]:
#@title #8.アプリケーションを立ち上げる

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"ngrok URL: {public_url}")
    app.run(port=5000)

ngrok URL: NgrokTunnel: "https://58ba-34-23-100-5.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 07:34:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 07:34:17] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 07:34:17] "GET /favicon.ico HTTP/1.1" 404 -
